In [3]:
from data_exploration import *

In [4]:
from imports import *

In [5]:
from collections import Counter

In [6]:
data_path = 'C:\\Users\\homol\\Documents\\diplomamunka\\adat\\tweets_2015_2020'

In [86]:
tweet_df_list = []
stock_df_list = []
symbol_list = ['TSLA','AAPL','GOOG','GOOGL','AMZN','MSFT']
for symbol in symbol_list:
    tweet_df, stock_df = load(data_path,symbol)
    tweet_df_list.append(tweet_df)
    stock_df_list.append(stock_df)

In [100]:
def logreturn(stock_df):
    stock_df = stock_df.rename(columns={"day_date": "date"})
    ordered_df = stock_df.sort_values('date')
    ordered_df['close_value'] = ordered_df['close_value'].apply(lambda x:np.log(x))
    diff_df = ordered_df[['close_value']].diff()
    stock_df['logreturn'] = diff_df['close_value']
    return stock_df

In [101]:
logreturn_df_list = []
for stock_df in stock_df_list:
    logreturn_df = logreturn(stock_df)
    logreturn_df_list.append(logreturn_df)

In [89]:
def get_confident_df(tweet_df):
    # reducing the dataframe only the neccessary cols
    max_value_df = tweet_df[['date','neutral','positive','negative']]
    # choosing the max value of the row 
    max_value_df['max_value'] = max_value_df[['neutral','positive','negative']].max(axis=1) 

    # the sentiment score is at least 0.8 
    confident_df = max_value_df.loc[max_value_df['max_value']>=0.8].reset_index(drop=True)
    #unconfident_df = max_value_df.loc[max_value_df['max_value'] < 0.8].reset_index(drop=True)

    # choosing the max value label
    confident_df['max_value_label'] = confident_df[['neutral','positive','negative']].idxmax(axis=1)
    #unconfident_df['max_value_label'] = unconfident_df[['neutral','positive','negative']].idxmax(axis=1)
    conf_df = confident_df.loc[(confident_df['date']>=datetime.date(2016,1,1)) &
                           (confident_df['date']<=datetime.date(2019,12,31))]
    return conf_df

In [90]:
confident_df_list = []
for tweet_df in tweet_df_list:
    confident_df = get_confident_df(tweet_df)
    confident_df_list.append(confident_df)

C:\Users\homol\AppData\Local\Temp/ipykernel_28752/3353066896.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  max_value_df['max_value'] = max_value_df[['neutral','positive','negative']].max(axis=1)


In [91]:
def get_ratio_df(conf_df):
    # conf:given_day_pos/given_day_all
    date_list = conf_df['date'].unique()
    pos_ratio_dict = {}
    neg_ratio_dict = {}
    for date in date_list:
        day_df = conf_df.loc[conf_df['date'] == date].reset_index(drop=True) 
        all_label = len(day_df)
        pos_label = len(day_df.loc[day_df['max_value_label'] == 'positive'].reset_index(drop=True))
        neg_label = len(day_df.loc[day_df['max_value_label'] == 'negative'].reset_index(drop=True))
        pos_ratio_dict[date] = pos_label/all_label
        neg_ratio_dict[date] = neg_label/all_label
        
    pos_ratio_df = pd.DataFrame(pos_ratio_dict.items(), columns=['date', 'pos_ratio'])
    neg_ratio_df = pd.DataFrame(neg_ratio_dict.items(), columns=['date', 'neg_ratio'])
    return pos_ratio_df, neg_ratio_df

In [92]:
def merge_ratio_df(pos_df,neg_df):
    sent_df = pd.merge(pos_df, neg_df,on='date')
    return sent_df

In [93]:
sent_df_list = []
for conf_df in confident_df_list:
    pos_ratio_df, neg_ratio_df = get_ratio_df(conf_df)
    sent_df = merge_ratio_df(pos_ratio_df,neg_ratio_df)
    sent_df_list.append(sent_df)

In [105]:
def merge_sent_logreturn(sent_df,logreturn_df):
    logreturn_sent_df = pd.merge(logreturn_df[['date','logreturn']],sent_df,on='date')
    return logreturn_sent_df

In [106]:
final_df_list = []
for i,sent_df in enumerate(sent_df_list):
    final_df = merge_sent_logreturn(sent_df,logreturn_df_list[i])
    final_df_list.append(final_df)

In [107]:
final_df_list[0]

,date,logreturn,pos_ratio,neg_ratio
0,2016-01-01,0.000000,0.134146,0.036585
1,2016-01-02,0.000000,0.037383,0.037383
2,2016-01-03,0.000000,0.147368,0.068421
3,2016-01-04,-0.071672,0.112832,0.075221
4,2016-01-05,0.000090,0.086466,0.041353
...,...,...,...,...
1456,2019-12-27,-0.001300,0.128418,0.046396
1457,2019-12-28,0.000000,0.098200,0.049100
1458,2019-12-29,0.000000,0.118966,0.075862
1459,2019-12-30,-0.037113,0.124013,0.082148


# Hipothesis test

In [ ]:
# KPSS test: H0:stationary (stationary of the series around deterministic trend)
# ADF test: H0: have unit root -> non stationary

In [108]:
from statsmodels.tsa.stattools import grangercausalitytests

In [109]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss

In [111]:
n_obs = 200
train_df_list = []
test_df_list = []
for final_df in final_df_list:
    df_train, df_test = final_df[0:-n_obs], final_df[-n_obs:]
    train_df_list.append(df_train)
    test_df_list.append(df_test)

In [117]:
def adf_test(df):
    result = adfuller(df.values)
    print('ADF Statistics: %f' % result[0])
    print('p-value: %f' % result[1])
    boolean = result[1] < 0.05
    print(f'Nullhipothesis rejected: {boolean}')
    print('Critical values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

In [113]:
col_list = ['logreturn','pos_ratio','neg_ratio']

In [120]:
company_name_list = ['Tesla', 'Apple', 'GOOG', 'GOOGL', 'Amazon', 'Microsoft']

In [121]:
for ind,train_df in enumerate(train_df_list):
    print(company_name_list[ind])
    for col in col_list:
        print(f'{col}\'s ADF test')
        adf_test(train_df[col])

Tesla
logreturn's ADF test
ADF Statistics: -35.713865
p-value: 0.000000
Nullhipothesis rejected: True
Critical values:
	1%: -3.436
	5%: -2.864
	10%: -2.568
pos_ratio's ADF test
ADF Statistics: -7.881956
p-value: 0.000000
Nullhipothesis rejected: True
Critical values:
	1%: -3.436
	5%: -2.864
	10%: -2.568
neg_ratio's ADF test
ADF Statistics: -4.333160
p-value: 0.000389
Nullhipothesis rejected: True
Critical values:
	1%: -3.436
	5%: -2.864
	10%: -2.568
Apple
logreturn's ADF test
ADF Statistics: -8.268742
p-value: 0.000000
Nullhipothesis rejected: True
Critical values:
	1%: -3.436
	5%: -2.864
	10%: -2.568
pos_ratio's ADF test
ADF Statistics: -5.877259
p-value: 0.000000
Nullhipothesis rejected: True
Critical values:
	1%: -3.436
	5%: -2.864
	10%: -2.568
neg_ratio's ADF test
ADF Statistics: -5.438217
p-value: 0.000003
Nullhipothesis rejected: True
Critical values:
	1%: -3.436
	5%: -2.864
	10%: -2.568
GOOG
logreturn's ADF test
ADF Statistics: -11.625374
p-value: 0.000000
Nullhipothesis rejecte

# P_value < 0.05 ,H0 is rejected->  stationary

In [122]:
def kpss_test(df):    
    statistic, p_value, n_lags, critical_values = kpss(df.values)
    
    print(f'KPSS Statistic: {statistic}')
    print(f'p-value: {p_value}')
    boolean = p_value < 0.05
    print(f'Nullhipothesis rejected: {boolean}')
    print(f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')

In [124]:
for ind,train_df in enumerate(train_df_list):
    print(company_name_list[ind])
    for col in col_list:
        print(f'{col}\'s KPSS test')
        kpss_test(train_df[col])

Tesla
logreturn's KPSS test
KPSS Statistic: 0.1272820760032258
p-value: 0.1
Nullhipothesis rejected: False
num lags: 23
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
pos_ratio's KPSS test
KPSS Statistic: 0.8582458417253541
p-value: 0.01
Nullhipothesis rejected: True
num lags: 23
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
neg_ratio's KPSS test
KPSS Statistic: 2.6342452806813794
p-value: 0.01
Nullhipothesis rejected: True
num lags: 23
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Apple
logreturn's KPSS test
KPSS Statistic: 0.06629974034106749
p-value: 0.1
Nullhipothesis rejected: False
num lags: 23
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
pos_ratio's KPSS test
KPSS Statistic: 0.5869475281894984
p-value: 0.023822951982772875
Nullhipothesis rejected: True
num lags: 23
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
neg_ratio's KPSS test
KPSS

C:\Users\homol\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1875: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)
C:\Users\homol\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1910: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(
C:\Users\homol\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1906: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warnings.warn(
C:\Use

In [130]:
transformed_train_df_list = []
for train_df in train_df_list:
    transformed_train_df = train_df.copy()
    transformed_train_df['pos_ratio'] = transformed_train_df['pos_ratio'].diff()
    transformed_train_df['neg_ratio'] = transformed_train_df['neg_ratio'].diff()
    transformed_train_df = transformed_train_df.dropna()
    transformed_train_df_list.append(transformed_train_df)

In [133]:
for ind,transformed_train_df in enumerate(transformed_train_df_list):
    print(company_name_list[ind])
    for col in col_list:
        print(f'{col}\'s KPSS test')
        kpss_test(transformed_train_df[col])

Tesla
logreturn's KPSS test
KPSS Statistic: 0.127367262241791
p-value: 0.1
Nullhipothesis rejected: False
num lags: 23
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
pos_ratio's KPSS test
KPSS Statistic: 0.013251743734908937
p-value: 0.1
Nullhipothesis rejected: False
num lags: 23
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
neg_ratio's KPSS test
KPSS Statistic: 0.01046328841808839
p-value: 0.1
Nullhipothesis rejected: False
num lags: 23
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Apple
logreturn's KPSS test
KPSS Statistic: 0.06636817156749479
p-value: 0.1
Nullhipothesis rejected: False
num lags: 23
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
pos_ratio's KPSS test
KPSS Statistic: 0.010259029463706971
p-value: 0.1
Nullhipothesis rejected: False
num lags: 23
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
neg_ratio's KPSS test
KPSS Statistic: 

C:\Users\homol\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1875: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)
C:\Users\homol\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1910: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(
C:\Users\homol\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1910: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(
C:\Use

# P_value > 0.05 , H0 is accepted -> series is stationary

It is always better to apply both the tests, so that it can be ensured that the series is truly stationary. Possible outcomes of applying these stationary tests are as follows:

<b>Case 1:</b> Both tests conclude that the series is not stationary - The series is not stationary <br>
<b>Case 2:</b> Both tests conclude that the series is stationary - The series is stationary<br>
<b>Case 3:</b> KPSS indicates stationarity and ADF indicates non-stationarity - The series is trend stationary. Trend needs to be removed to make series strict stationary. The detrended series is checked for stationarity.<br>
<b>Case 4:</b> KPSS indicates non-stationarity and ADF indicates stationarity - The series is difference stationary. Differencing is to be used to make series stationary. The differenced series is checked for stationarity.<br>
Here, due to the difference in the results from ADF test and KPSS test, it can be inferred that the series is trend stationary and not strict stationary. The series can be detrended by differencing or by model fitting.

In [138]:
from statsmodels.tsa.api import VAR

for ind,transformed_train_df in enumerate(transformed_train_df_list):
    print(company_name_list[ind])
    model = VAR(transformed_train_df[['logreturn','pos_ratio','neg_ratio']])
    for i in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]:
        result = model.fit(i)
        aic_list = []
        aic_list.append(result.aic)
    print(f'min AIC{min(aic_list)}, Lag Order: {aic_list.index(min(aic_list))+1}')

Tesla
min AIC-22.050487820816098, Lag Order: 1
Apple
min AIC-23.117389679893993, Lag Order: 1
GOOG
min AIC-21.959440914438, Lag Order: 1
GOOGL
min AIC-22.255287995101774, Lag Order: 1
Amazon


C:\Users\homol\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\homol\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\homol\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\homol\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\homol\anaconda3\lib\site-packages\statsmode

min AIC-22.806399064060056, Lag Order: 1
Microsoft
min AIC-22.671602760720592, Lag Order: 1


C:\Users\homol\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'


In [85]:
from statsmodels.tsa.stattools import grangercausalitytests

maxlag=1
test = 'ssr_chi2test'

def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
   
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

,log_return_x,pos_ratio_x
log_return_y,1.0000,0.0505
pos_ratio_y,0.0016,1.0000


In [144]:
gr_df_list = []
for ind,transformed_train_df in enumerate(transformed_train_df_list):
    print(company_name_list[ind])
    gr_df = grangers_causation_matrix(transformed_train_df[['logreturn','pos_ratio','neg_ratio']], 
                              variables = transformed_train_df[['logreturn','pos_ratio','neg_ratio']].columns)
    gr_df_list.append(gr_df)

Tesla
Apple
GOOG
GOOGL
Amazon
Microsoft


In [149]:
for ind,gr_df in enumerate(gr_df_list):
    print(company_name_list[ind])
    print(f'{gr_df}','\n')
    save_path = '\\Users\\homol\\Desktop\\gr_df_'+company_name_list[ind]+'.xlsx'
    gr_df.to_excel(save_path)

Tesla
             logreturn_x  pos_ratio_x  neg_ratio_x
logreturn_y       1.0000       0.0514       0.2786
pos_ratio_y       0.0029       1.0000       0.0005
neg_ratio_y       0.0157       0.0007       1.0000 

Apple
             logreturn_x  pos_ratio_x  neg_ratio_x
logreturn_y          1.0       0.0005       0.0000
pos_ratio_y          0.0       1.0000       0.0003
neg_ratio_y          0.0       0.0032       1.0000 

GOOG
             logreturn_x  pos_ratio_x  neg_ratio_x
logreturn_y       1.0000       0.0010       0.0275
pos_ratio_y       0.0267       1.0000       0.1582
neg_ratio_y       0.0028       0.3685       1.0000 

GOOGL
             logreturn_x  pos_ratio_x  neg_ratio_x
logreturn_y       1.0000       0.0106       0.0031
pos_ratio_y       0.1051       1.0000       0.0610
neg_ratio_y       0.0002       0.0001       1.0000 

Amazon
             logreturn_x  pos_ratio_x  neg_ratio_x
logreturn_y       1.0000       0.0083       0.0022
pos_ratio_y       0.0122       1.0000       

In [150]:
0.01<0.05

True